In [3]:
# import libraries

import numpy as np
import pandas as pd
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from datetime import datetime
import glob
import shutil
import glob
import pickle
import os

In [2]:
class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


In [7]:
file_list = glob.glob('Base_Prototipo/*')
file_list.sort()

In [8]:
df = pd.DataFrame()
target = [10,11]

# Initializing progress bar

bar = ProgBar(len(file_list),'\nConverting PDF to LaTex...')

for i,f in enumerate(file_list):

    if i == 0:    

        data = np.genfromtxt(f, delimiter=',')

        if i in target: y = np.ones(len(data)).reshape(-1,1)

        else: y = np.zeros(len(data)).reshape(-1,1)

        data  = np.hstack((data,y))

        pickle.dump( data, open( "Arquivos_Processados/{}.pkl".format(i), "wb" ), protocol=4 )

        os.remove(f) 


    else:

        new_data = np.genfromtxt(f, delimiter=',')
        
        if i in target: y = np.ones(len(new_data)).reshape(-1,1)

        else: y = np.zeros(len(new_data)).reshape(-1,1)

        new_data  = np.hstack((new_data,y))

        data = np.vstack((data,new_data))

        pickle.dump( new_data, open( "Arquivos_Processados/{}.pkl".format(i), "wb" ), protocol=4 )

        os.remove(f) 

    bar.update()

print("\nSaving Input...")

pickle.dump( data, open( "Input.pkl", "wb" ), protocol=4 )

print("\nInput saved.")

#del data



Converting PDF to LaTex...
[--------------------------------------------------] 100.00% - 27 of 27
Saving Input...

Input saved.


In [7]:
np.savetxt("data.csv",data,delimiter=',')

In [8]:
del data


In [6]:
df.shape

(0, 0)

In [9]:
# Gambiarra pra rodar só os dados de um dia
matheus = []
thiago = []
for f in file_list:
    dia = f.split('/')[1].split('-')[0]
    
    if dia == '18':
        matheus.append(f)
    else:
        thiago.append(f)

file_list = thiago # modificar essa linha para rodar o dia 19

In [12]:
df = pd.DataFrame()

for f in file_list:
    
    new_name = "Arquivos_Processados/" + f.split('/')[-1]

    data = np.genfromtxt(f, delimiter=',')
    
    df2 = pd.DataFrame(data, columns=['id', 'time', 
                                     'Sensor_0', 'Sensor_1',
                                     'Sensor_2', 'Sensor_3',
                                     'Sensor_4', 'Sensor_5'])
    
    inicio = datetime.now()

    extracted_features = extract_features(df2, column_id="id", column_sort="time",
                                default_fc_parameters = EfficientFCParameters(),
                                chunksize=100,
                                n_jobs=4,
                                disable_progressbar=False)
    
    df = pd.concat([df, extracted_features])
    
    print('--->', f)
    print(datetime.now() - inicio)
    print(' ')
    df.to_csv('extracted_features.csv')

    shutil.move(f, new_name)
    
    
df = df.dropna(axis='columns')
df.to_csv('extracted_features.csv')

Feature Extraction:   0%|          | 0/40 [00:00<?, ?it/s]

'Arquivos_Processados/19-Jan-2022-08.47.14.csv'